In [25]:
import socket
import re

class HTTPRequestBuilder:
    def __init__(self):
        self._request = ""
        self._method = "GET"
        self._path = "/"
        self._query_parameters = {}
        self._version = "HTTP/1.1"
        self._headers = {"Host":"example.com"}
        self._response_chunk_length = 1024
        self._port = 80
        self._body = ""

    def handle_method(self):
        if self._method.upper() in ["GET"] and self._body != "":
            raise Exception(f"{self._method.upper()} method cannot have a body")
        
        if self._method.upper() in ["POST", "PUT","DELETE"] :
            self._headers["Content-Length"] = len(self._body)
            

    def build(self) -> HTTPRequestBuilder:
        self.handle_method()
        self._request = f"{self._method} {self._path}"

        if not self._query_parameters == {}:
            self._request += "?"
            for param in self._query_parameters:
                self._request += f"{param}={self._query_parameters[param]}&"
            self._request = self._request[0:-1]
        
        self._request +=  f" {self._version}\r\n"
        
        for key in self._headers:
            self._request +=  f"{key}: {self._headers[key]}\r\n"

        self._request += "\r\n"
        self._request += self._body
        return self

    def set_method(self, method:str) -> HTTPRequestBuilder:
        if method.upper() not in ["GET","POST","PUT","DELETE"]:
            raise Exception("Invalid Method")
        self._method = method.upper()
        return self
    
    def set_header(self, key:str, value:str) -> HTTPRequestBuilder:
        self._headers[key] = value
        return self
    
    def set_port(self, port:int) -> HTTPRequestBuilder:
        self._port = port
        return self

    def set_response_chunk_length(self, len:int) -> HTTPRequestBuilder:
        self._response_chunk_length = len
        return self

    def set_body(self, body:str) -> HTTPRequestBuilder:
        self._body = body
        return self

    def set_path(self, path:str) -> HTTPRequestBuilder:
        regex = re.compile(r"^(?:/(?:[A-Za-z0-9\-._~!$&'()*+,;=:@]|%(?:[A-Fa-f0-9]{2}))*)+$")
        if not regex.match(path):
            raise Exception("Invalid Path")
        self._path = path
        return self
        
    def set_version(self, version:str) -> HTTPRequestBuilder:
        regex = re.compile(r"^HTTP\/(1\.0|1\.1|2|3)$")
        if not regex.match(version):
            raise Exception("Invalid HTTP Version")
        self._version = version
        return self

    def add_query_parameter(self, param, value) -> HTTPRequestBuilder:
        self._query_parameters[param] = value
        return self

    def send(self):
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
            sock.connect((self._headers["Host"],self._port))
            sock.send(self._request.encode())
            chunk = sock.recv(self._response_chunk_length)
            response = b""
            while (chunk):
                response += chunk
                chunk = sock.recv(self._response_chunk_length)
        return response


    def print(self):
        print(self._request)

In [34]:
import eventlet
pool = eventlet.GreenPool()

def set_and_send(request:HTTPRequestBuilder, i:int):
    #print("----------------------\\")
    request.set_method("POST").set_header("Host","127.0.0.1").set_path('/tr').set_version("HTTP/1.1")
    request.set_port(60000).set_body('{"username":"Alice'+'","age": '+str(i)+'}').set_header("Content-Type", "application/json").set_header("Connection","close")
    request.build()
    response = ""
    response = request.send().decode()
    if(-1 == response.find("'"+str(i)+"'") ):
        request.print()
        print("\nResponse:")
        print(response)
        print()
    #print("----------------------/\n")



requests = []
for _ in range(300):
    requests.append(HTTPRequestBuilder())

i = 0
for request in requests:
    pool.spawn(set_and_send, request, i)
    i +=1
pool.waitall()





Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.14/lib/python3.14/site-packages/eventlet/hubs/hub.py", line 471, in fire_timers
    timer()
    ~~~~~^^
  File "/Library/Frameworks/Python.framework/Versions/3.14/lib/python3.14/site-packages/eventlet/hubs/timer.py", line 59, in __call__
    cb(*args, **kw)
    ~~^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.14/lib/python3.14/site-packages/eventlet/greenthread.py", line 272, in main
    result = function(*args, **kwargs)
  File "/var/folders/7g/n1vctl6n4cj5sd0bdx89yms00000gn/T/ipykernel_4427/2520509184.py", line 10, in set_and_send
    response = request.send().decode()
               ~~~~~~~~~~~~^^
  File "/var/folders/7g/n1vctl6n4cj5sd0bdx89yms00000gn/T/ipykernel_4427/710015638.py", line 87, in send
    chunk = sock.recv(self._response_chunk_length)
ConnectionResetError: [Errno 54] Connection reset by peer
